In [1]:
import sys
sys.path.insert(0, '/home/vipul/llm/iterate')
print(sys.path)

['/home/vipul/llm/iterate', '/home/vipul/miniconda3/envs/llm/lib/python310.zip', '/home/vipul/miniconda3/envs/llm/lib/python3.10', '/home/vipul/miniconda3/envs/llm/lib/python3.10/lib-dynload', '', '/home/vipul/miniconda3/envs/llm/lib/python3.10/site-packages']


In [35]:
from dataset import JSONLDataset, TabularDataset, PickleDataset
import models.openai as openai
from util import parse_example, parse_tsv_example, parse_qaner_example, score_sets
import numpy as np
import time
import re

from dotenv import load_dotenv

import argparse
import random
from tqdm.auto import tqdm
import os, pdb
import json
import shutil
import logging
from datetime import datetime
import signal

In [87]:
def run_prompt(prompt, model):
    completion = model.complete(prompt)
    if completion is None or completion == '':
        return "Could not complete prompt"
    return completion

def construct_prompt(srcfile, promptfile, egs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}]', end=' ')
        print()

def get_prompt_eg_acc(srcfile, eg_idxs):
    ds = JSONLDataset(srcfile)
    egs = []
    for idx in eg_idxs:
        egs.append(ds[idx])
    
    for eg in egs:
        for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
            if ptok == gtok:
                print(ptok, end=' ')
            else:
                print(f'[{ptok}->{gtok}]', end=' ')
        print()
    
def create_model():
    load_dotenv('../../.env')
    openai.setup_api_key(os.environ.get('OPENAI_API_KEY'))

    model_args = openai.ChatGPT.DEFAULT_ARGS
    model_args['engine'] = 'gpt-4'
    model_args['max_tokens'] = 768
    model_args['request_timeout'] = 200
    return openai.ChatGPT(model_args)

def parse_response(response):
    lines = response.strip().split('\n')
    pred = [a.split('\t') for a in lines if '\t' in a]
    return list(zip(*pred))

def load_jsonl(file):
    lines = open(file, 'r').readlines()
    data = [json.loads(line.strip()) for line in lines]
    return data

In [90]:
def load_egs(directory, eg_prefix):
    egs = load_jsonl(directory + f"/responses.json")
    
    tgt_eg = None
    for eg in egs:
        if eg['input'].startswith(eg_prefix):
            tgt_eg = eg
            break

    if not tgt_eg:
        print('Could not find example')
        return

    print(f"input: {tgt_eg['input']}\n")
    print(f"Examples: [wrong->correct] \n")
    
    exemplars = tgt_eg['examples']
    for e in exemplars:
        a, b = list(zip(*[a.rsplit('_') for a in e.split(' ')]))
        a_str = ' '.join(a)
        for i, eg in enumerate(egs):
            if eg['input'] == a_str:
                print(i, end=" ")
                print(a_str, end="  \n")
                lbls = []
                acc = 0
                for (ptok, gtok) in zip(eg['pred_labels'], eg['gold_labels']):
                    if ptok == gtok:
                        lbls.append(ptok)
                        acc += 1
                    else:
                        lbls.append(f'[{ptok}->{gtok}]')
                print(f"{100*acc/len(a):.2f} {' '.join(lbls)}", end="  \n\n")
                break

def construct_prompt(directory, eg_prefix, gold=False):
    egs = load_jsonl(directory + f"/responses.json")
    
    prompt = []
    tgt_eg = None
    for eg in egs:
        if eg['input'].startswith(eg_prefix):
            tgt_eg = eg
            break

    if not tgt_eg:
        print('Could not find example')
        return

    prompt.append(f"Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:")
    prompt.append("")

    exemplars = tgt_eg['examples']
    for e in exemplars:
        a, b = list(zip(*[a.rsplit('_') for a in e.split(' ')]))
        a_str = ' '.join(a)
        prompt.append(f'Sentence: {a_str}')
        prompt.append('Tags:\n```')
        if gold:
            for i, eg in enumerate(egs):
                if eg['input'] == a_str:
                    for (word, lbl) in zip(eg['input'].split(" "), eg['gold_labels']):
                        prompt.append(f'{word}\t{lbl}')
                    prompt.append('```')
                    prompt.append("")
                    break
        else:
            for (word, lbl) in zip(a, b):
                prompt.append(f'{word}\t{lbl}')
            prompt.append('```')
            prompt.append("")
    
    prompt.append(f'Sentence: {tgt_eg["input"]}')
    prompt.append(f'Tags:\n```\n')
    
    return "\n".join(prompt)

In [ ]:
print(construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', 'Ds Gueten'))

In [20]:
eg_1_idxs = [66, 58, 93, 45, 15, 68, 26, 70]

In [120]:
def find_and_replace_n_with_regex(text, target_word, replacement_word, n):
    pattern = r'\b{}\b'.format(target_word)
    count = 0
    def replacer(match):
        nonlocal count
        count += 1
        if count != 1 and count <= (n+1): # don't replace the occurence of the tag in the instruction!
            return replacement_word
        else:
            return match.group(0)

    return re.sub(pattern, replacer, text)

In [121]:
find_and_replace_n_with_regex('fish fish fish fish fish', 'fish', 'bear', 4)

'fish bear bear bear bear'

In [122]:
model = create_model()

In [125]:
def print_results(prompt, model):
    tags = parse_response(run_prompt(prompt, model))
    model.cleanup()
    print(' '.join(tags[0]))
    print(' '.join(tags[1]))

In [126]:
# 11 AUX in eg 1
for i in [5, 11]:
    prompt = construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', 'Ds Gueten')
    prompt = find_and_replace_n_with_regex(prompt, 'AUX', 'VERB', i)
    print(prompt)
    print_results(prompt, model)
    print()

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .
Tags:
```
I	PRON
main	VERB
,	PUNCT
das	DET
Ganze	NOUN
letscht	ADJ
Wuchä	NOUN
isch	VERB
mier	PRON
scho	ADV
ächli	ADV
iigfaarä	VERB
.	PUNCT
```

Sentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .
Tags:
```
Du	PRON
gsehsch	VERB
uus	PRON
,	PUNCT
wi	SCONJ
wenn	SCONJ
de	DET
nöime	ADJ
no	ADV
hättisch	VERB
z	PART
trinken	VERB
übercho	VERB
.	PUNCT
```

Sentence: Dir weit mer doch nid verzöue , di Wäutsche heige vo eim Tag uf en anger ufghört Chuttlen ässe .
Tags:
```
Dir	PRON
weit	VERB
mer	PRON
doch	ADV
nid	ADV
verzöue	VERB
,	PUNCT
di	DET
Wäutsche	NOUN
heige	VERB
vo	ADP
eim	DET
Tag	NOUN
uf	ADP
en	DET
anger	ADJ
ufghört	VERB
Chuttlen	NOUN
ässe	VERB
.	PUNCT


In [127]:
prompt = construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', 'Ds Gueten', gold=True)
print(prompt)
print_results(prompt, model)
print()

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .
Tags:
```
I	PRON
main	VERB
,	PUNCT
das	DET
Ganze	NOUN
letscht	ADJ
Wuchä	NOUN
isch	AUX
mier	PRON
scho	ADV
ächli	ADV
iigfaarä	VERB
.	PUNCT
```

Sentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .
Tags:
```
Du	PRON
gsehsch	VERB
uus	PART
,	PUNCT
wi	CCONJ
wenn	SCONJ
de	PRON
nöime	ADV
no	ADV
hättisch	AUX
z	DET
trinken	NOUN
übercho	VERB
.	PUNCT
```

Sentence: Dir weit mer doch nid verzöue , di Wäutsche heige vo eim Tag uf en anger ufghört Chuttlen ässe .
Tags:
```
Dir	PRON
weit	AUX
mer	PRON
doch	ADV
nid	PART
verzöue	VERB
,	PUNCT
di	DET
Wäutsche	NOUN
heige	AUX
vo	ADP
eim	DET
Tag	NOUN
uf	ADP
en	DET
anger	ADJ
ufghört	VERB
Chuttlen	NOUN
ässe	VERB
.	PUNCT
```

Backing off 0.4 seconds after 2 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPART\n,\tPUNCT\nwi\tCCONJ\nwenn\tSCONJ\nde\tPRON\nnöime\tADV\nno\tADV\nhättisch\tAUX\nz\tDET\ntrinken\tNOUN\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Backing off 4.6 seconds after 4 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I main , das Ganze letscht Wuchä isch mier scho ächli iigfaarä .\nTags:\n```\nI\tPRON\nmain\tVERB\n,\tPUNCT\ndas\tDET\nGanze\tNOUN\nletscht\tADJ\nWuchä\tNOUN\nisch\tAUX\nmier\tPRON\nscho\tADV\nächli\tADV\niigfaarä\tVERB\n.\tPUNCT\n```\n\nSentence: Du gsehsch uus , wi wenn de nöime no hättisch z trinken übercho .\nTags:\n```\nDu\tPRON\ngsehsch\tVERB\nuus\tPART\n,\tPUNCT\nwi\tCCONJ\nwenn\tSCONJ\nde\tPRON\nnöime\tADV\nno\tADV\nhättisch\tAUX\nz\tDET\ntrinken\tNOUN\nübercho\tVERB\n.\tPUNCT\n```\n\nSentence: Dir weit mer doch nid verzöue , d

Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .
DET NOUN AUX ADV AUX PUNCT SCONJ PRON ADJ ADV ADJ AUX AUX PUNCT SCONJ PRON ADV AUX AUX PART VERB PUNCT



In [128]:
# 7 AUX in eg 2
for i in [3, 7]:
    prompt = construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', "D ' Geb" )
    prompt = find_and_replace_n_with_regex(prompt, 'AUX', 'VERB', i)
    print(prompt)
    print_results(prompt, model)
    print()

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .
Tags:
```
I	PRON
ha	VERB
so	ADV
fescht	ADJ
mit	ADP
der	DET
flache	ADJ
Hang	NOUN
uf	ADP
dä	DET
Beizetisch	NOUN
ghoue	VERB
,	PUNCT
dass	SCONJ
grad	ADV
d	DET
Gleser	NOUN
hei	ADV
aafo	VERB
waggele	VERB
.	PUNCT
```

Sentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .
Tags:
```
Dä	PRON
hends	VERB
mittlerwiile	ADV
z	ADP
'	PUNCT
Luzern	PROPN
entlòò	VERB
,	PUNCT
will	SCONJ
är	PRON
zvill	ADV
Gool	NOUN
gschòssä	VERB
hätt	VERB
und	CCONJ
wäg	ADP
dem	DET
dä	DET
Luzerner	NOUN
irgendwiä	ADV
unhaimlich	ADJ
wòrdän	VERB
isch	VERB
.	PUNCT
```

Sentence: Ufere Tafle st

Backing off 1.2 seconds after 2 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .\nTags:\n```\nI\tPRON\nha\tVERB\nso\tADV\nfescht\tADJ\nmit\tADP\nder\tDET\nflache\tADJ\nHang\tNOUN\nuf\tADP\ndä\tDET\nBeizetisch\tNOUN\nghoue\tVERB\n,\tPUNCT\ndass\tSCONJ\ngrad\tADV\nd\tDET\nGleser\tNOUN\nhei\tADV\naafo\tVERB\nwaggele\tVERB\n.\tPUNCT\n```\n\nSentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .\nTags:\n```\nDä\tPRON\nhends\tVERB\nmittl

Backing off 1.1 seconds after 4 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .\nTags:\n```\nI\tPRON\nha\tVERB\nso\tADV\nfescht\tADJ\nmit\tADP\nder\tDET\nflache\tADJ\nHang\tNOUN\nuf\tADP\ndä\tDET\nBeizetisch\tNOUN\nghoue\tVERB\n,\tPUNCT\ndass\tSCONJ\ngrad\tADV\nd\tDET\nGleser\tNOUN\nhei\tADV\naafo\tVERB\nwaggele\tVERB\n.\tPUNCT\n```\n\nSentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .\nTags:\n```\nDä\tPRON\nhends\tVERB\nmittl

D ' Gebäude ufem Firmagländ vu Energizer in La Chaux-de-Fonds ( NE ) sind zrugbaut worda .
DET PUNCT NOUN ADP NOUN ADP PROPN ADP PROPN PUNCT PROPN PUNCT AUX VERB VERB PUNCT

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .
Tags:
```
I	PRON
ha	VERB
so	ADV
fescht	ADJ
mit	ADP
der	DET
flache	ADJ
Hang	NOUN
uf	ADP
dä	DET
Beizetisch	NOUN
ghoue	VERB
,	PUNCT
dass	SCONJ
grad	ADV
d	DET
Gleser	NOUN
hei	ADV
aafo	VERB
waggele	VERB
.	PUNCT
```

Sentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .
Tags:
```
Dä	PRON
hends	VERB
mittlerwiile	ADV
z	ADP
'	PUNCT
Luzern	PROPN
entlòò	VERB
,	PUNCT
will	SCONJ
är	PRON
zvill

In [129]:
prompt = construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', "D ' Geb", gold=True)
print(prompt)
print_results(prompt, model)
print()

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .
Tags:
```
I	PRON
ha	AUX
so	ADV
fescht	ADJ
mit	ADP
der	DET
flache	ADJ
Hang	NOUN
uf	ADP
dä	DET
Beizetisch	NOUN
ghoue	VERB
,	PUNCT
dass	SCONJ
grad	ADV
d	DET
Gleser	NOUN
hei	AUX
aafo	VERB
waggele	VERB
.	PUNCT
```

Sentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .
Tags:
```
Dä	PRON
hends	AUX
mittlerwiile	ADV
z	ADP
'	PUNCT
Luzern	PROPN
entlòò	VERB
,	PUNCT
will	SCONJ
är	PRON
zvill	DET
Gool	NOUN
gschòssä	VERB
hätt	AUX
und	CCONJ
wäg	ADP
dem	PRON
dä	DET
Luzerner	PROPN
irgendwiä	ADV
unhaimlich	ADJ
wòrdän	AUX
isch	AUX
.	PUNCT
```

Sentence: Ufere Tafle staht

Backing off 1.5 seconds after 2 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .\nTags:\n```\nI\tPRON\nha\tAUX\nso\tADV\nfescht\tADJ\nmit\tADP\nder\tDET\nflache\tADJ\nHang\tNOUN\nuf\tADP\ndä\tDET\nBeizetisch\tNOUN\nghoue\tVERB\n,\tPUNCT\ndass\tSCONJ\ngrad\tADV\nd\tDET\nGleser\tNOUN\nhei\tAUX\naafo\tVERB\nwaggele\tVERB\n.\tPUNCT\n```\n\nSentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .\nTags:\n```\nDä\tPRON\nhends\tAUX\nmittler

Backing off 7.8 seconds after 4 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': "Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha so fescht mit der flache Hang uf dä Beizetisch ghoue , dass grad d Gleser hei aafo waggele .\nTags:\n```\nI\tPRON\nha\tAUX\nso\tADV\nfescht\tADJ\nmit\tADP\nder\tDET\nflache\tADJ\nHang\tNOUN\nuf\tADP\ndä\tDET\nBeizetisch\tNOUN\nghoue\tVERB\n,\tPUNCT\ndass\tSCONJ\ngrad\tADV\nd\tDET\nGleser\tNOUN\nhei\tAUX\naafo\tVERB\nwaggele\tVERB\n.\tPUNCT\n```\n\nSentence: Dä hends mittlerwiile z ' Luzern entlòò , will är zvill Gool gschòssä hätt und wäg dem dä Luzerner irgendwiä unhaimlich wòrdän isch .\nTags:\n```\nDä\tPRON\nhends\tAUX\nmittler

D ' Gebäude ufem Firmagländ vu Energizer in La Chaux-de-Fonds ( NE ) sind zrugbaut worda .
DET PUNCT NOUN ADP NOUN ADP PROPN ADP PROPN PUNCT PROPN PUNCT AUX VERB AUX PUNCT



In [130]:
# 14 AUX in eg 3
for i in [7, 14]:
    prompt = construct_prompt('../new_results/gpt4_new/germanic_new/8t/gsw', "I cha der ihri" )
    prompt = find_and_replace_n_with_regex(prompt, 'AUX', 'VERB', i)
    print(prompt)
    print_results(prompt, model)
    print()

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .
Tags:
```
I	PRON
ha	VERB
ar	PART
Marie-Claire	PROPN
gseit	VERB
,	PUNCT
es	PRON
sig	VERB
mer	PRON
chli	ADV
schlächt	ADJ
und	CCONJ
i	PRON
mög	VERB
jetz	ADV
nümm	ADV
liire	VERB
.	PUNCT
```

Sentence: De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .
Tags:
```
De	DET
Spanier	NOUN
hed	VERB
de	DET
Kontakt	NOUN
vermettlet	VERB
,	PUNCT
d	DET
Rumäne	NOUN
sölled	VERB
d	DET
Holländer	PROPN
ombrocht	VERB
ha	VERB
.	PUNCT
```

Sentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .
Tags:
```
Ds	DET
Gueten	NOUN
isch	VERB
immerhin	ADV
gsi	VERB
,	PUNCT
dass	SCONJ
i	PRON
ungerdesse	A

Backing off 0.2 seconds after 2 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .\nTags:\n```\nI\tPRON\nha\tVERB\nar\tPART\nMarie-Claire\tPROPN\ngseit\tVERB\n,\tPUNCT\nes\tPRON\nsig\tVERB\nmer\tPRON\nchli\tADV\nschlächt\tADJ\nund\tCCONJ\ni\tPRON\nmög\tVERB\njetz\tADV\nnümm\tADV\nliire\tVERB\n.\tPUNCT\n```\n\nSentence: De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .\nTags:\n```\nDe\tDET\nSpanier\tNOUN\nhed\tVERB\nde\tDET\nKontakt\tNOUN\nvermettlet\tVERB\n,\tPUNCT\nd\tDET\nRumäne\tNOUN\nsölled\tVERB\nd\tD

Backing off 7.5 seconds after 4 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:\n\nSentence: I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .\nTags:\n```\nI\tPRON\nha\tVERB\nar\tPART\nMarie-Claire\tPROPN\ngseit\tVERB\n,\tPUNCT\nes\tPRON\nsig\tVERB\nmer\tPRON\nchli\tADV\nschlächt\tADJ\nund\tCCONJ\ni\tPRON\nmög\tVERB\njetz\tADV\nnümm\tADV\nliire\tVERB\n.\tPUNCT\n```\n\nSentence: De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .\nTags:\n```\nDe\tDET\nSpanier\tNOUN\nhed\tVERB\nde\tDET\nKontakt\tNOUN\nvermettlet\tVERB\n,\tPUNCT\nd\tDET\nRumäne\tNOUN\nsölled\tVERB\nd\tD

I cha der ihri Telefonnummere gä , de nimmsch mou unverbindlech Kontakt uuf .
PRON AUX PRON PRON NOUN VERB PUNCT PRON VERB ADV ADJ NOUN ADP PUNCT

Tag the following sentence according to the Part of Speech (POS) of each word. The valid tags are ADJ, ADP, ADV, AUX, CCONJ, DET, INTJ, NOUN, NUM, PART, PRON, PROPN, PUNCT, SCONJ, SYM, VERB, X. Follow the format specified in the examples below:

Sentence: I ha ar Marie-Claire gseit , es sig mer chli schlächt und i mög jetz nümm liire .
Tags:
```
I	PRON
ha	VERB
ar	PART
Marie-Claire	PROPN
gseit	VERB
,	PUNCT
es	PRON
sig	VERB
mer	PRON
chli	ADV
schlächt	ADJ
und	CCONJ
i	PRON
mög	VERB
jetz	ADV
nümm	ADV
liire	VERB
.	PUNCT
```

Sentence: De Spanier hed de Kontakt vermettlet , d Rumäne sölled d Holländer ombrocht ha .
Tags:
```
De	DET
Spanier	NOUN
hed	VERB
de	DET
Kontakt	NOUN
vermettlet	VERB
,	PUNCT
d	DET
Rumäne	NOUN
sölled	VERB
d	DET
Holländer	PROPN
ombrocht	VERB
ha	VERB
.	PUNCT
```

Sentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed

I cha der ihri Telefonnummere gä , de nimmsch mou unverbindlech Kontakt uuf .
PRON VERB PRON PRON NOUN VERB PUNCT DET VERB ADV ADJ NOUN ADP PUNCT



In [135]:
prompt = """Given the following sentence in swiss german, list out all the words which are verbs (words tagged as VERB in POS tagging). Do NOT mark auxiliary verbs or adverbs as verbs. return ONLY the verbs in the sentence.

Sentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .
Words: """
print(prompt)
output = run_prompt(prompt, model)
model.cleanup()
print(output)

Given the following sentence in swiss german, list out all the words which are verbs (words tagged as VERB in POS tagging). Do NOT mark auxiliary verbs or adverbs as verbs. return ONLY the verbs in the sentence.

Sentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .
Words: 
Backing off 0.9 seconds after 1 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given the following sentence in swiss german, list out all the words which are verbs (words tagged as VERB in POS tagging). Do NOT mark auxiliary verbs or adverbs as verbs. return ONLY the verbs in the sentence.\n\nSentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .\nWords: '}], {'engine': 'gpt-4', 'max_tokens': 768, 'stop': None, 'temperature': 0.0, 'request_timeout': 200}) and kwargs {'chat': Tr

Backing off 26.7 seconds after 6 tries calling function <function get_response at 0x7f08d3df3d90> with args ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given the following sentence in swiss german, list out all the words which are verbs (words tagged as VERB in POS tagging). Do NOT mark auxiliary verbs or adverbs as verbs. return ONLY the verbs in the sentence.\n\nSentence: Ds Gueten isch immerhin gsi , dass i ungerdesse söfu müed bi gsi , dass i ändlech ha chönne go schlofe .\nWords: '}], {'engine': 'gpt-4', 'max_tokens': 768, 'stop': None, 'temperature': 0.0, 'request_timeout': 200}) and kwargs {'chat': True}
{'target': <function get_response at 0x7f08d3df3d90>, 'args': ([{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Given the following sentence in swiss german, list out all the words which are verbs (words tagged as VERB in POS tagging). Do NOT mark auxiliary verbs or adverbs as verbs. re